In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn.functional as F

from models import model, tokenizer, device
from synth import cross_entropy_given

/home/tuccio/synth/menv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tuccio/synth/menv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
Y = """ thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Giovanni told me that his favourite pizza is the capricciosa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""
prompt = f"""{Y}::INTRODUCE::"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
output = model.generate(
    input.input_ids,
    attention_mask=input.attention_mask,
    max_length=len(input.input_ids[0])+6,
    num_return_sequences=100,
    no_repeat_ngram_size=2,
    do_sample=True,
    temperature=1
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
Y = """Generate 5 examples of a Y sentence and a X sentence containing 6 tokens that reduce the Y sentence cross-entropy.
Use the following format:
<y> Y sentence here </y>
<x> X sentence here </x>"""
prompt = f"""{Y}"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
output = model.generate(
    input.input_ids,
    attention_mask=input.attention_mask,
    max_length=len(input.input_ids[0])+600,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    do_sample=True,
    temperature=1
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [29]:
prompt = f"""\n"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
output = model.generate(
    input.input_ids,
    attention_mask=input.attention_mask,
    max_length=len(input.input_ids[0])+6,
    num_return_sequences=100,    
    no_repeat_ngram_size=2,
    do_sample=True,
    temperature=1
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [35]:
for out in output:
    print("\n-----------------\n"+tokenizer.decode(out, skip_special_tokens=True))


-----------------
 thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Giovanni told me that his favourite pizza is the capricciosa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say.::INTRODUCE::
But they also produce these

-----------------
 thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Giovanni told me that his favourite pizza is the capricciosa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say.::INTRODUCE:: He told about his recipe from

-----------------
 thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Giovanni told me that his favourite pizza is the capricciosa with salsiccia. 
When I was a kid, I used to watch t

In [36]:
results = []
for i in output:
    answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
    results.append((
        cross_entropy_given(answ, Y, diff=True).item(),
        answ
        ))
    
results.sort()

In [37]:
results

[(-0.10944151878356934, ' This pizza makes me happy at'),
 (-0.1053459644317627, ' The pizza I make from sa'),
 (-0.10464739799499512, '\nHis Pizza is just awesome'),
 (-0.10369992256164551, '\nHere are his favorite pizza'),
 (-0.10185098648071289, '\nI have always liked pizza'),
 (-0.09585189819335938, '\nI liked this pizza because'),
 (-0.08196568489074707, ' This pizza was delicious\nThe'),
 (-0.07778811454772949, "\nHere's some amazing pizza"),
 (-0.07446789741516113, 'He never made me believe pizza'),
 (-0.07327079772949219, '.\nThis pizzade was'),
 (-0.06531238555908203, '\nTaste of pizza here'),
 (-0.0649869441986084, '\nThis is actually more Italian'),
 (-0.06099724769592285, ' When I started eating pizza I'),
 (-0.06087827682495117, "\nThe pizza makers' pizz"),
 (-0.05710649490356445, '\nThese are good Italian pizz'),
 (-0.05694937705993652, 'He used the name Italian pizza'),
 (-0.05366802215576172, '\nI never felt so old'),
 (-0.05301642417907715, '\nI like the sauce more'),


In [17]:
phrases = [
    "Pizza is the most delicious", 
    "Getting pizza is my favorite",
    "Eating pizza is my favorite",
    "Making pizza is my favorite",
    "I believe the best Italian", 
    "Let me tell you something", 
    "In my opinion, the tastiest", 
    "I must say, the greatest"
    ]

for p in phrases:
    print(f"{p}: \t {cross_entropy_given(p, Y, diff=True).item()}")




Pizza is the most delicious: 	 -0.16765403747558594
Getting pizza is my favorite: 	 -0.2020125389099121
Eating pizza is my favorite: 	 -0.22490191459655762
Making pizza is my favorite: 	 -0.22872066497802734
I believe the best Italian: 	 -0.11588573455810547
Let me tell you something: 	 0.028350114822387695
In my opinion, the tastiest: 	 -0.06910991668701172
I must say, the greatest: 	 -0.01048421859741211
